In [1]:
from __future__ import division
import numpy as np
import PyDSTool as dst

#Investigating the Rössler attractor using `PyDSTool` and `mpl3d`

I'll start off by presenting a function that is nice to have in many scenarios, as it neatly bundles up the creation of a `PyDSTool` dynamical system.

In [ ]:
def build_sys(gentype, sys_name, pars, varspecs, ics, 
              xname, yname, zname, intdomain, 
              tdata=[0, 50], algparams=None, events=[], tdomain=None):
    # we must give a name
    DSargs = dst.args(name=sys_name)

    # parameters
    DSargs.pars = pars

    # rhs of the differential equation
    DSargs.varspecs = varspecs

    # initial conditions
    DSargs.ics = ics

    # set the domain of integration.
    DSargs.xdomain = intdomain
    
    if tdomain != None:
        print "Setting generator tdomain: {}".format(tdomain)
        DSargs.tdomain = [0, 1000]
    else:
        print "Setting generator tdata: {}".format(tdata)
        DSargs.tdata = tdata

    # to avoid typos / bugs, use built-in Symbolic differentation!
    f = [DSargs.varspecs[xname], DSargs.varspecs[yname], DSargs.varspecs[zname]]
    Df = dst.Diff(f, [xname, yname, zname])
    print "Jacobian: ", str(Df.renderForCode())
    DSargs.fnspecs = {'Jacobian': (['t', xname, yname, zname],
                                   str(Df.renderForCode()))}

    if algparams != None:
        DSargs.algparams = algparams

    # Make auxiliary functions to define event lines near saddle
    res = pp.make_distance_to_line_auxfn('Gamma_out_plus',
                                         'Gamma_out_plus_fn',
                                         (xname, yname), True)
    man_pars = res['pars']
    man_auxfns = res['auxfn']
    res = pp.make_distance_to_line_auxfn('Gamma_out_minus',
                                         'Gamma_out_minus_fn',
                                         (xname, yname), True)
    man_pars.extend(res['pars'])
    man_auxfns.update(res['auxfn'])

    # update param values with defaults (0)
    for p in man_pars:
        DSargs.pars[p] = 0

    if gentype in [dst.Generator.Vode_ODEsystem, dst.Generator.Euler_ODEsystem]:
        targetlang = 'python'
    else:
        targetlang = 'c'

    DSargs.fnspecs.update(man_auxfns)
    ev_plus = dst.Events.makeZeroCrossEvent(expr='Gamma_out_plus_fn(%s,%s)' % (xname, yname),
                                            dircode=0,
                                            argDict={'name': 'Gamma_out_plus',
                                                     'eventtol': 1e-5,
                                                     'eventdelay': 1e-3,
                                                     'starttime': 0,
                                                     'precise': False,
                                                     'active': False,
                                                     'term': True},
                                            targetlang=targetlang,
                                            varnames=[xname, yname],
                                            fnspecs=man_auxfns,
                                            parnames=man_pars
                                            )
    ev_minus = dst.Events.makeZeroCrossEvent(expr='Gamma_out_minus_fn(%s,%s)' % (xname, yname),
                                             dircode=0,
                                             argDict={'name': 'Gamma_out_minus',
                                                      'eventtol': 1e-5,
                                                      'eventdelay': 1e-3,
                                                      'starttime': 0,
                                                      'precise': False,
                                                      'active': False,
                                                      'term': True},
                                             targetlang=targetlang,
                                             varnames=[xname, yname],
                                             fnspecs=man_auxfns,
                                             parnames=man_pars
                                             )

    DSargs.events = [ev_plus, ev_minus] + events
    print "Generating ODE system..."
    return gentype(DSargs), DSargs.fnspecs['Jacobian']